In [70]:
# Run in python console
import nltk; 

# Run in terminal or command prompt

In [71]:
import re
import numpy as np
import pandas as pd
from pprint import pprint


In [72]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [73]:
import spacy

In [74]:
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [75]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [76]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [77]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [78]:
# Import Dataset
# df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
# df = pd.read_json('../data/DocumentCloud/categorize/chicago.json')
# print(df.target_names.unique())
# df
# df['content'][0]
# df.head()

In [79]:
from os import listdir
import os
from os.path import isfile, join
import json
files = json.load(open("../data/DocumentCloud/filtered_documents.json"))
mypath = '../data/DocumentCloud/interested files'
# onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
# print(onlyfiles[0])
fileNum = 0
data = []
# print('hah')
for f in files:
    if f['source']!='Chicago City Council':
        continue
    if fileNum>50:
        break
    
    filename = f['id']+".txt"
    filepath = join(mypath,filename)
    b = os.path.getsize(filepath)
    if b<10240: 
        continue
    with open(filepath, 'r',encoding="utf-8") as file:
        d = file.read().replace('\xa0',' ')
        d = d.replace('\n',' ')
    data.append(d)
    fileNum+=1
#     print(b)

In [80]:
data

['1ül$*fÊl  -$  VT Ftr l, h?  \'i  COMMITTEE ON PEDESTRIAN & TRAFFIC SAFETY FINAL AGENDA OF MATTERS TO BE CONSIDERED  !ì  December 12,2019, 12:00pm, Room 201A  DIRECT INTRODUCTIONS  -  ITEM  WARD  PARKING PROHIBITED AT ALL TIMES  1  1  867 North Marshfield Avenue, Disabled Permit 109879 1C2019-89291  2  1  1720 North Mozart Street, Disabled Permit 1 19683 tO2019-89301  3  I  1921 North Central Park Avenue, Disabled Permit 119685 102019-89311  4  2  1416 North Bosworth Avenue, Disabled Permit 1 13185 102019-89321  5  5  6936 South Crandon Avenue, Disabled Permit 1 13691 [O2019-8933]  6  5  6901 South Crandon Avenue, Disabled Permit 113700102019-89341  7  5  71 16  B  5  6803 South Crandon Avenue, Disabled Permit 1187731C2019-89361  9  5  5335 South Dorchester Avenue, Disabled Permit 1 18786 [o2019-8937]  10  5  5126 South lngleside Avenue, Disabled Permit 1 18783 1O2019-B93BI  11  5  5207 South lngleside Avenue, Disabled Permit 118677 102019-89391  12  5  5212 South Cornell Avenue, Disa

In [81]:
# Convert to list
# data = df.content.values.tolist()
# type(data[0])
# Remove Emails
# data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# # Remove new line characters
# data = [re.sub('\s+', ' ', sent) for sent in data]

# # Remove distracting single quotes
# data = [re.sub("\'", "", sent) for sent in data]

# pprint(data[:1])
# data

In [82]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['ul', 'fel', 'vt', 'ftr', 'committee', 'on', 'pedestrian', 'traffic', 'safety', 'final', 'agenda', 'of', 'matters', 'to', 'be', 'considered', 'december', 'pm', 'room', 'direct', 'introductions', 'item', 'ward', 'parking', 'prohibited', 'at', 'all', 'times', 'north', 'marshfield', 'avenue', 'disabled', 'permit', 'north', 'mozart', 'street', 'disabled', 'permit', 'to', 'north', 'central', 'park', 'avenue', 'disabled', 'permit', 'north', 'bosworth', 'avenue', 'disabled', 'permit', 'south', 'crandon', 'avenue', 'disabled', 'permit', 'south', 'crandon', 'avenue', 'disabled', 'permit', 'south', 'crandon', 'avenue', 'disabled', 'permit', 'south', 'dorchester', 'avenue', 'disabled', 'permit', 'south', 'lngleside', 'avenue', 'disabled', 'permit', 'bi', 'south', 'lngleside', 'avenue', 'disabled', 'permit', 'south', 'cornell', 'avenue', 'disabled', 'permit', 'south', 'champlain', 'avenue', 'disabled', 'permit', 'south', 'lndiana', 'avenue', 'disabled', 'permit', 'west', 'nd', 'place', 'disabled

In [83]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['ul', 'fel', 'vt', 'ftr', 'committee', 'on', 'pedestrian_traffic_safety', 'final', 'agenda', 'of', 'matters', 'to', 'be', 'considered', 'december', 'pm', 'room', 'direct_introductions_item', 'ward', 'parking', 'prohibited', 'at', 'all', 'times', 'north', 'marshfield', 'avenue', 'disabled', 'permit', 'north', 'mozart', 'street', 'disabled', 'permit', 'to', 'north', 'central', 'park', 'avenue', 'disabled', 'permit', 'north', 'bosworth', 'avenue', 'disabled', 'permit', 'south', 'crandon', 'avenue', 'disabled', 'permit', 'south', 'crandon', 'avenue', 'disabled', 'permit', 'south', 'crandon', 'avenue', 'disabled', 'permit', 'south', 'dorchester', 'avenue', 'disabled', 'permit', 'south', 'lngleside', 'avenue', 'disabled', 'permit', 'bi', 'south', 'lngleside', 'avenue', 'disabled', 'permit', 'south', 'cornell', 'avenue', 'disabled', 'permit', 'south', 'champlain', 'avenue', 'disabled', 'permit', 'south', 'lndiana', 'avenue', 'disabled', 'permit', 'west', 'nd', 'place', 'disabled', 'permit', 

In [84]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [85]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

['safety', 'final', 'agenda', 'matter', 'consider', 'pm', 'room', 'parking', 'prohibit', 'disable', 'disable', 'permit', 'disable', 'disable', 'disable', 'disable', 'disable', 'disable', 'disable', 'permit', 'disable', 'disable', 'disable', 'disable', 'place', 'disabled', 'disable', 'disabled', 'disable', 'disable', 'disable', 'permit', 'disable', 'disable', 'permit', 'disabled', 'permit', 'item', 'parking', 'prohibit', 'time', 'disabled', 'disabled', 'disable', 'disable', 'disable', 'disable', 'disable', 'disable', 'disable', 'disable', 'disable', 'disable', 'disable', 'disable', 'disable', 'disable', 'disable', 'permit', 'disable', 'disable', 'disable', 'disable', 'disable', 'permit', 'disabled', 'disable', 'disable', 'permit', 'disabled', 'permit', 'parking', 'prohibit', 'time', 'disabled', 'place', 'disable', 'permit', 'place', 'disabled', 'permit', 'disabled', 'disable', 'permit', 'place', 'disabled', 'permit', 'disable', 'street', 'disable', 'permit', 'disabled', 'permit', 'disab

In [86]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])
# corpus

[[(0, 2), (1, 1), (2, 1), (3, 3), (4, 16), (5, 12), (6, 8), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 6), (13, 1), (14, 1), (15, 1), (16, 15), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 270), (24, 151), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 33), (36, 1), (37, 3), (38, 1), (39, 1), (40, 1), (41, 8), (42, 1), (43, 12), (44, 1), (45, 1), (46, 27), (47, 2), (48, 1), (49, 1), (50, 9), (51, 1), (52, 1), (53, 1), (54, 3), (55, 8), (56, 2), (57, 9), (58, 1), (59, 1), (60, 2), (61, 1), (62, 12), (63, 1), (64, 1), (65, 20), (66, 69), (67, 7), (68, 1), (69, 162), (70, 1), (71, 24), (72, 12), (73, 20), (74, 1), (75, 6), (76, 20), (77, 7), (78, 1), (79, 17), (80, 16), (81, 1), (82, 3), (83, 38), (84, 2), (85, 29), (86, 1), (87, 1), (88, 2), (89, 3), (90, 1), (91, 1), (92, 18), (93, 27), (94, 8), (95, 2), (96, 1), (97, 5), (98, 24), (99, 15), (100, 15), (101, 3), (102, 1), (103, 1), (104, 1), (105, 38), (106, 1), (107, 20)

In [87]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('add', 2),
  ('agenda', 1),
  ('alltime', 1),
  ('also', 3),
  ('amend', 16),
  ('approach', 12),
  ('benefit', 8),
  ('bus', 1),
  ('central', 1),
  ('charter', 1),
  ('cleaning', 1),
  ('collector_roadway', 1),
  ('commercial', 6),
  ('concern', 1),
  ('consider', 1),
  ('court', 1),
  ('day', 15),
  ('dead', 1),
  ('deming', 1),
  ('departments_recommendation', 1),
  ('designate', 1),
  ('dicken', 1),
  ('disabl', 1),
  ('disable', 270),
  ('disabled', 151),
  ('end', 1),
  ('esl', 1),
  ('establish', 1),
  ('feel', 1),
  ('file', 1),
  ('final', 1),
  ('first', 1),
  ('flashing_light', 1),
  ('fo', 1),
  ('food', 2),
  ('foot', 33),
  ('green', 1),
  ('halste', 3),
  ('hayne', 1),
  ('hermitage', 1),
  ('holiday', 1),
  ('hour', 8),
  ('include', 1),
  ('insert', 12),
  ('instal', 1),
  ('intersection', 1),
  ('item', 27),
  ('keate', 2),
  ('leave', 1),
  ('lnserte', 1),
  ('loading', 9),
  ('major', 1),
  ('maplewood', 1),
  ('matter', 1),
  ('may', 3),
  ('meter', 8),
  ('mob

In [88]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [89]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.011*"ill" + 0.007*"iir" + 0.006*"will" + 0.006*"agreement" + '
  '0.006*"lnmate" + 0.006*"llt" + 0.004*"seek" + 0.004*"vehicle" + '
  '0.004*"part" + 0.004*"rule"'),
 (1,
  '0.071*"public" + 0.068*"premise" + 0.066*"adjacent" + 0.065*"way" + '
  '0.065*"know" + 0.061*"maintain" + 0.057*"right" + 0.042*"construct" + '
  '0.021*"sign" + 0.018*"project"'),
 (2,
  '0.000*"premise" + 0.000*"right" + 0.000*"maintain" + 0.000*"public" + '
  '0.000*"adjacent" + 0.000*"know" + 0.000*"way" + 0.000*"construct" + '
  '0.000*"ordinance" + 0.000*"unit"'),
 (3,
  '0.000*"premise" + 0.000*"public" + 0.000*"way" + 0.000*"right" + '
  '0.000*"know" + 0.000*"adjacent" + 0.000*"construct" + 0.000*"maintain" + '
  '0.000*"district" + 0.000*"purpose"'),
 (4,
  '0.000*"refer" + 0.000*"address" + 0.000*"unit" + 0.000*"district" + '
  '0.000*"common" + 0.000*"purpose" + 0.000*"building" + 0.000*"document" + '
  '0.000*"ordinance" + 0.000*"owner"'),
 (5,
  '0.075*"public" + 0.071*"way" + 0.068*"know" 

In [90]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -4.484753893821857

Coherence Score:  0.49279333184005003


In [91]:
# Visualize the topics
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
# vis

In [92]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.011*"ill" + 0.007*"iir" + 0.006*"will" + 0.006*"agreement" + '
  '0.006*"lnmate" + 0.006*"llt" + 0.004*"seek" + 0.004*"vehicle" + '
  '0.004*"part" + 0.004*"rule"'),
 (1,
  '0.071*"public" + 0.068*"premise" + 0.066*"adjacent" + 0.065*"way" + '
  '0.065*"know" + 0.061*"maintain" + 0.057*"right" + 0.042*"construct" + '
  '0.021*"sign" + 0.018*"project"'),
 (2,
  '0.000*"premise" + 0.000*"right" + 0.000*"maintain" + 0.000*"public" + '
  '0.000*"adjacent" + 0.000*"know" + 0.000*"way" + 0.000*"construct" + '
  '0.000*"ordinance" + 0.000*"unit"'),
 (3,
  '0.000*"premise" + 0.000*"public" + 0.000*"way" + 0.000*"right" + '
  '0.000*"know" + 0.000*"adjacent" + 0.000*"construct" + 0.000*"maintain" + '
  '0.000*"district" + 0.000*"purpose"'),
 (4,
  '0.000*"refer" + 0.000*"address" + 0.000*"unit" + 0.000*"district" + '
  '0.000*"common" + 0.000*"purpose" + 0.000*"building" + 0.000*"document" + '
  '0.000*"ordinance" + 0.000*"owner"'),
 (5,
  '0.075*"public" + 0.071*"way" + 0.068*"know" 

In [93]:
# for i,topic in lda_model.show_topics(formatted=True, num_topics=20, num_words=10):
#     print(str(i)+": "+ topic)
#     print()

In [94]:
topic_id = lda_model[corpus[0]][0][0][0] # corpus[0] means the first document.
text = lda_model.print_topics()[topic_id][1]
keywords = re.findall(r'"([^"]*)"', text)

print(keywords)

['disable', 'permit', 'disabled', 'parking', 'zone', 'repeal', 'foot', 'thereof', 'stop', 'sign']


In [95]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [96]:
en_stop = set(nltk.corpus.stopwords.words('english'))
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [97]:
all_topics = lda_model.get_document_topics(corpus, per_word_topics=True)
print(len(all_topics))
for doc_topics, word_topics, phi_values in all_topics:
    print('New Document \n')
    print('Document topics:', doc_topics)
#     print('Word topics:', word_topics)
#     print('Phi values:', phi_values)
    print(" ")
    print('-------------- \n')

51
New Document 

Document topics: [(7, 0.99973935)]
 
-------------- 

New Document 

Document topics: [(12, 0.99988)]
 
-------------- 

New Document 

Document topics: [(10, 0.9997556)]
 
-------------- 

New Document 

Document topics: [(10, 0.999614)]
 
-------------- 

New Document 

Document topics: [(10, 0.9997556)]
 
-------------- 

New Document 

Document topics: [(10, 0.9997121)]
 
-------------- 

New Document 

Document topics: [(5, 0.020331478), (10, 0.97381717)]
 
-------------- 

New Document 

Document topics: [(7, 0.9994754)]
 
-------------- 

New Document 

Document topics: [(5, 0.8870863), (7, 0.07615423), (12, 0.03671601)]
 
-------------- 

New Document 

Document topics: [(7, 0.999496)]
 
-------------- 

New Document 

Document topics: [(10, 0.9997696)]
 
-------------- 

New Document 

Document topics: [(5, 0.10986466), (12, 0.88137096)]
 
-------------- 

New Document 

Document topics: [(5, 0.0500968), (12, 0.9497739)]
 
-------------- 

New Document 

Docu

In [98]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [99]:
from os import listdir
import os
from os.path import isfile, join
mypath = '../data/DocumentCloud/interested files'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
fileNum = 0

for file in onlyfiles:
    data = []
    if fileNum<52:
        fileNum += 1
        continue
    if fileNum>100:
        break
    filepath = os.path.join(mypath,file)
    with open(filepath, 'r',encoding="utf-8") as file:
        d = file.read().replace('\xa0',' ')
        d = d.replace('\n',' ')
    data.append(d)
    fileNum += 1
#     print(fileNum)
# file = '5664106-Zoning-Board-of-Appeals-2016-12-16-Minutes.txt'
# filepath = os.path.join(mypath,file)
# with open(filepath, 'r',encoding="utf-8") as file:
#     d = file.read().replace('\xa0',' ')
#     d = d.replace('\n',' ')
# data.append(d)
    data_words = list(sent_to_words(data))
    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner'])

    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)

    # Create Corpus
    texts = data_lemmatized

    corpus = [id2word.doc2bow(text) for text in texts]
    all_topics = lda_model.get_document_topics(corpus, per_word_topics=True)
    print(len(all_topics))
    for doc_topics in all_topics:
        print('New Document: ', file)
        print('Document topics:', doc_topics[0])
    #     print('Word topics:', word_topics)
    #     print('Phi values:', phi_values)
        print(" ")
        print('-------------- \n')

1
New Document:  <_io.TextIOWrapper name='../data/DocumentCloud/interested files\\5015910-Board-of-Health-Meeting-2018-10-17-Agenda.txt' mode='r' encoding='utf-8'>
Document topics: [(7, 0.8856393), (10, 0.09675519)]
 
-------------- 

1
New Document:  <_io.TextIOWrapper name='../data/DocumentCloud/interested files\\5015911-Zoning-Board-of-Appeals-2016-02-19-Minutes.txt' mode='r' encoding='utf-8'>
Document topics: [(5, 0.022123339), (7, 0.45727137), (8, 0.029203355), (12, 0.46407267), (15, 0.027205613)]
 
-------------- 

1
New Document:  <_io.TextIOWrapper name='../data/DocumentCloud/interested files\\5015912-Zoning-Board-of-Appeals-2015-07-17-Minutes.txt' mode='r' encoding='utf-8'>
Document topics: [(1, 0.026876595), (7, 0.37653938), (8, 0.04384575), (10, 0.046414737), (12, 0.5062057)]
 
-------------- 

1
New Document:  <_io.TextIOWrapper name='../data/DocumentCloud/interested files\\5015913-Zoning-Board-of-Appeals-2015-12-18-Map.txt' mode='r' encoding='utf-8'>
Document topics: [(7, 

1
New Document:  <_io.TextIOWrapper name='../data/DocumentCloud/interested files\\5015947-Quality-and-Patient-Safety-Committee-Meetings.txt' mode='r' encoding='utf-8'>
Document topics: [(0, 0.032334674), (1, 0.051655926), (2, 0.031038284), (3, 0.031063823), (4, 0.030249959), (5, 0.06853782), (6, 0.03057796), (7, 0.13630663), (8, 0.039147068), (9, 0.031054232), (10, 0.12829739), (11, 0.029842027), (12, 0.13818988), (13, 0.03120866), (14, 0.030084118), (15, 0.035651363), (16, 0.03001161), (17, 0.03236581), (18, 0.03132882), (19, 0.031053988)]
 
-------------- 

1
New Document:  <_io.TextIOWrapper name='../data/DocumentCloud/interested files\\5015948-Zoning-Board-of-Appeals-2015-03-20-Minutes.txt' mode='r' encoding='utf-8'>
Document topics: [(5, 0.19254887), (7, 0.3208158), (10, 0.096206225), (12, 0.39021838)]
 
-------------- 

1
New Document:  <_io.TextIOWrapper name='../data/DocumentCloud/interested files\\5015949-Electoral-Board-Suburban-Cook-2018-04-24-April.txt' mode='r' encoding='u

In [100]:
import re 
s = '"adfkjakldf"jdaljlkfja"liadlkjf"'
re.findall(r'"([^"]*)"', s)

['adfkjakldf', 'liadlkjf']